In [303]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time, os
import pickle

In [304]:
# jobs = {}

In [349]:
indeed_uk_url = "https://uk.indeed.com/viewjob?jk="
indeed_us_url = "https://www.indeed.com/viewjob?jk="

In [350]:
def scrape(soup, job):
   description = soup.find(id="jobDescriptionText")
   if description:
      job["description"] = description.get_text()
   else:
      time.sleep(20)
      job["description"] = soup.find(id="jobDescriptionText").get_text()

   metadata = soup.find("div", {"class": "jobsearch-JobMetadataHeader-item"})
   if metadata:
      job["metadata"] = metadata.get_text("|")

   title = soup.find("div", {"class": "jobsearch-JobInfoHeader-title-container"})
   if title:
      job["title"] = title.get_text()

   companyInfo = soup.find("div", {"class": "jobsearch-CompanyInfoContainer"})
   if companyInfo:
      job["companyInfo"] = companyInfo.get_text("|")

   return job

In [351]:
def scrape_details(driver, url, job, job_id):
   driver.get(url)
   soup = bs(driver.page_source)

   try:
      job = scrape(soup, job)
   except Exception as e:
      print(f"Error scraping. Job id: {job_id}")
      return job, (job_id, "scraping", e)

   return job, None

In [352]:
def scrape_job_details(driver, job_ids, jobs):
   incomplete_scrapes = []
   already_scraped = []

   counter = 0

   deleted = ['b0f82946b85ea228', '9de3cf82db801fdf']

   for job_id in job_ids.keys() :
      if (job_id not in jobs) and (job_id not in deleted):
         job = {}
         job['id'] = job_id

         url = indeed_uk_url + job_id
         try:
            counter += 1
            job, incomplete_scrape = scrape_details(driver, url, job, job_id)
            if incomplete_scrape:
               incomplete_scrapes.append(incomplete_scrape)
            else:
               jobs[job_id] = job
         except Exception as e:
            incomplete_scrapes.append((job_id, "navigating", e))
      else:
         already_scraped.append(job_id)

      # if counter > 3:
      #    break

   driver.quit()
   return jobs, incomplete_scrapes, already_scraped

In [353]:
# Load data
with open('job_ids.pickle', 'rb') as handle:
   job_ids = pickle.load(handle)

job_ids

{'264f504fa90d856f': None,
 '5573e1b6447a8af6': None,
 '41ce545464f94ca7': None,
 'c461ed010a20c835': None,
 '1fe7173e0d89af7a': None,
 'f1b98d776e72be61': None,
 'ec3849d4dda665c6': None,
 '278142c7efa8c49a': None,
 '8e4ebf6f8f6568f8': None,
 'c8610c368721c0ab': None,
 '98f78125259351fb': None,
 '9a36d2355241924b': None,
 'f11971796d62ded9': None,
 '1619eda6ee3dcbb0': None,
 '14a3bb31248f543a': None,
 'e7d30ae416f248ca': None,
 '9da03cdc1f38e23e': None,
 '488bf3e35de656b4': None,
 '2f632b10cfb5bf93': None,
 '62f378db55687030': None,
 '561da9499d1ccc7f': None,
 '7a7c63c336e285aa': None,
 'e7362c51c4671308': None,
 '0edc5a7b195f5c79': None,
 '1e75c64559d233ac': None,
 '7d06f30f27ef44b9': None,
 'de7ad63840a4e83f': None,
 'af9df3033edaa0ad': None,
 '14cc06ce96650681': None,
 '4b91fb225034dbcd': None,
 'c727cfe03669a9fe': None,
 'a1bc538f4c7ea3e3': None,
 '70332396f3c8444b': None,
 '5886011685afc18c': None,
 '3c5304799384cdf7': None,
 'd201512b8f8050e7': None,
 '81fa085584b780c0': None,
 

In [354]:
len(job_ids)

717

In [355]:
chromedriver = "/Users/william.bell/opt/anaconda3/envs/metis/lib/python3.8/site-packages/chromedriver_binary/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(3)

In [356]:
start = time.time()
cars, incomplete_scrapes, already_scraped = scrape_job_details(driver, job_ids, jobs)
end = time.time()
print(f"Time taken: {end - start}" )

Time taken: 26.270166873931885


In [357]:
incomplete_scrapes

[]

In [358]:
len(incomplete_scrapes)

0

In [359]:
len(already_scraped)

669

In [360]:
len(jobs)

715

In [361]:
jobs

{'264f504fa90d856f': {'id': '264f504fa90d856f',
  'description': 'Live shows make us feel good. They’re a time to hang with our friends, discover new artists or lose ourselves on a dancefloor. We’re on a mission to bring all of this to more fans, more often – and that’s where you come in.\nWe’re looking for a Data Scientist to join our Data Science teamand help build products to improve the experience of fans, partners and stakeholders.\nAt DICE, you’ll be part of the company that’s redefining live entertainment. It’s a place where you can be yourself, influence the culture, and create work that you’re proud of.\n\nAbout the role:\nIn close collaboration with our Data Analytics team, Project Managers and Designers, you will perform internal analysis of key strategic initiatives across multiple departments. Your insight and recommendations will be converted into data products that will reinforce internal tooling and the predictive abilities of the team as a whole.\nAlongside forming rep

In [362]:
# with open('jobs_indeed_uk.pickle', 'wb') as handle:
#    pickle.dump(jobs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [363]:
# Load data
# with open('jobs_indeed_usa.pickle', 'rb') as handle:
#    unserialized_data = pickle.load(handle)
#
# unserialized_data